<a href="https://colab.research.google.com/github/grace3999/USV_Python/blob/colab/2_xr_Annotations_from_netcdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#mount google drive containings required files: 1) csv of annotation features, 2) netcdf files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install netcdf4

    100% |████████████████████████████████| 3.9MB 8.0MB/s 
    100% |████████████████████████████████| 307kB 32.3MB/s 


In [0]:
import numpy as np
import pandas as pd
import os
import re
from scipy import stats
import xarray as xr

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [0]:
def create_annot_dataset(path, annot_data, size_random):
    """takes in path of netcdf file (created using notebook 1) and corresponding annotations (created using notebook 0)
    and creates a new data set of only slices correspondingn to annotations (and random noise if set to True)"""
    
    #get animal name from file (select based on naming scheme)
    #name = re.search("\d\d\d", path).group(0)
    #print(name)
    
    name, session = path.split('/')[-1].split('-')[1], path.split('/')[-1].split('-')[0]
    print(name, session)
    
    #open xarray Dataset
    data = xr.open_dataset(path)
    print(data['slices'].shape)
    
    #get time_stamps of animal's annotations, select slices and save
    yes = annot_data[(annot_data['animal_number'] ==  name) & (annot_data['session'] ==  session)]['time_stamp'].sort_values().values
    data_yes = data.sel(slices = yes)
    
    #get slice indexs (e.g. time_stamps) that are not annotations, select slices and save
    slice_indexes = data['slices'].values
    no = np.setdiff1d(slice_indexes,yes)
    print(yes.shape[0] + no.shape[0])
    no_short = np.random.choice(no, size_random, replace=False)
    data_no = data.sel(slices = no_short)
    
    return data_yes, data_no

In [0]:
def compute_spectral_features(Dataset):
    """takes in netcdf dataset and computes 8 features"""
    
    spec_power = []
    spec_purs = []
    spec_centroid = []
    spec_spread = []
    spec_skewness = []
    spec_kurtosis = []
    spec_slope = []
    spec_roll_off = []
    
    freq_array = Dataset['freq'].values
    
    #compute power sum using groupby
    spec_power = Dataset.groupby('slices').sum(xr.ALL_DIMS)['__xarray_dataarray_variable__'].values
    
    #compute other features for each slice individually
    for value in Dataset['slices'].values:
        
        spec_pur = stats.gmean(Dataset.sel(slices = value)['__xarray_dataarray_variable__'].values, axis = None) / Dataset.sel(slices = value)['__xarray_dataarray_variable__'].values.mean()
        
        mag_array = Dataset['__xarray_dataarray_variable__'].sel(slices=value).max(dim = 'times').values
        mag_probs = mag_array/sum(mag_array)
        freq_mag = freq_array*mag_probs
        
        spec_cent = sum(freq_mag)
        spec_spr = np.var(freq_mag)
        spec_skew = stats.skew(freq_mag)
        spec_kurt = stats.kurtosis(freq_mag)
        slope, intercept, r_value, p_value, std_err = stats.linregress(freq_array, freq_mag)
        spec_ro = .95*sum(freq_mag)
        
        spec_purs.append(spec_pur)
        spec_centroid.append(spec_cent)
        spec_spread.append(spec_spr)
        spec_skewness.append(spec_skew)
        spec_kurtosis.append(spec_kurt)
        spec_slope.append(slope)
        spec_roll_off.append(spec_ro)
        
    return spec_power, spec_purs, spec_centroid, spec_spread, spec_skewness, spec_kurtosis, spec_slope, spec_roll_off

In [0]:
def create_annotation_slice_features(path, annot_data, size_random = 50):
    """takes in path of netcdf file (created using notebook 1), corresponding annotations (created using notebook 0),
    and size desired for random slicecs from each file.
    uses create_annot_dataset and compute_spectral_features functions to create a new data set of computed features
    for only slices correspondingn to annotations and random noise"""
    
    
    #get animal name from path (select based on naming scheme)
    #name = re.search("\d\d\d", path).group(0)
    
    name, session = path.split('/')[-1].split('-')[1], path.split('/')[-1].split('-')[0]
    print(name, session)
    
    #create datasets of slices of known annotations and a random selection of noise
    data_yes, data_no = create_annot_dataset(path, annot_data, size_random)
    
    #compute spectral features
    yes_spec_power, yes_spec_purs, yes_spec_centroid, yes_spec_spread, yes_spec_skewness, yes_spec_kurtosis, yes_spec_slope, yes_spec_roll_off = compute_spectral_features(data_yes)
    no_spec_power, no_spec_purs, no_spec_centroid, no_spec_spread, no_spec_skewness, no_spec_kurtosis, no_spec_slope, no_spec_roll_off = compute_spectral_features(data_no)

    #add computed features to exisiting dataframe of known annotations
    annot_yes = annot_data[(annot_data['animal_number'] ==  name) & (annot_data['session'] ==  session)].sort_values(by=['time_stamp'])
    annot_yes['power_sum'] = yes_spec_power
    annot_yes['spec_pur'] = yes_spec_purs
    annot_yes['spec_cent'] = yes_spec_centroid
    annot_yes['spec_spread'] = yes_spec_spread
    annot_yes['spec_skew'] = yes_spec_skewness
    annot_yes['spec_kurt'] = yes_spec_kurtosis
    annot_yes['spec_slope'] = yes_spec_slope
    annot_yes['spec_roll'] = yes_spec_roll_off
    
    #create and fill dataframe for randomly selected noise slices
    annot_no = pd.DataFrame(columns = ['animal_number', 'session', 'time_stamp', 'Annotation'], index = np.arange(0,size_random))
    annot_no['animal_number'] = name
    annot_no['session'] = session
    annot_no['Annotation'] = 'rand_noise'
    annot_no['time_stamp'] = data_no['slices'].values
    annot_no['power_sum'] = no_spec_power
    annot_no['spec_pur'] = no_spec_purs
    annot_no['spec_cent'] = no_spec_centroid
    annot_no['spec_spread'] = no_spec_spread
    annot_no['spec_skew'] = no_spec_skewness
    annot_no['spec_kurt'] = no_spec_kurtosis
    annot_no['spec_slope'] = no_spec_slope
    annot_no['spec_roll'] = no_spec_roll_off
    
    return annot_yes, annot_no 

Create data frame of annotation info

In [0]:
#annot_path_cage = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_cagepair.csv'
#annot_path_CPApost = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_CPApair.csv'
#annot_paths = [annot_path_CPApost, annot_path_cage]

annot_path_pain = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_pain.csv'

In [0]:
annot = pd.read_csv(annot_path_pain)
annot_data = pd.DataFrame(data = annot)
annot_data['Annotation'] = ['BBC'] * annot_data['Annotation'].shape[0]
print(annot_data.shape)
annot_data.head()

(178, 6)


,Unnamed: 0,animal_number,session,time_stamp,Annotation,High Freq (Hz)
0,0,A2,D2,32692.5,BBC,48403.2
1,1,A2,D2,97515.0,BBC,50048.4
2,2,A2,D2,124942.5,BBC,41465.2
3,3,A2,D2,292027.5,BBC,48718.6
4,4,A2,D2,296505.0,BBC,52345.3


In [0]:
annot_data = pd.DataFrame()

for path in annot_path_pain:
    annot = pd.read_csv(path)
    annot = pd.DataFrame(data = annot)
    annot = annot[annot['Annotation'] != 'radar']
    print(annot.shape)
    print(annot.Annotation.value_counts())
    
    annot_data = annot_data.append(annot)

print(annot_data.shape)
annot_data.head()

ParserError: ignored

Find path names for each netcdf file corresponding to wav file that has annotated data

In [0]:
netcdf_path_CPApost = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/CPA_post'

netcdf_path_pain = annot_path_pain = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Pain'

In [0]:
path_names = []
files = os.listdir(netcdf_path_pain)
for file in files: 
        path_names.append(netcdf_path_pain + "/" + file)

len(path_names)

18

In [121]:
annot_features_yes

,animal_number,session,time_stamp,Annotation,High Freq (Hz),power_sum,spec_pur,spec_cent,spec_spread,spec_skew,spec_kurt,spec_slope,spec_roll
0,A2,D2,32692.5,BBC,48403.2,33001.679688,0.278688,35066.527850,44975.041894,2.700772,6.672909,-0.001613,33313.201458
1,A2,D2,97515.0,BBC,50048.4,42889.351562,0.209728,33325.032145,64997.459077,2.717043,6.765939,-0.002337,31658.780537
2,A2,D2,124942.5,BBC,41465.2,32306.705078,0.250006,30640.698606,49617.524980,2.513463,5.396313,-0.002205,29108.663676
3,A2,D2,292027.5,BBC,48718.6,128251.039062,0.089902,37497.496525,201737.430661,3.751878,12.999226,-0.002475,35622.621699
4,A2,D2,296505.0,BBC,52345.3,65412.070312,0.180102,35595.230449,62452.123418,3.204285,11.323622,-0.002025,33815.468927
5,A4,D2,75330.0,BBC,34805.4,37319.996094,0.230533,27045.172317,72826.665154,4.731524,24.274570,-0.002163,25692.913701
6,A4,D2,121005.0,BBC,36614.4,59058.792969,0.179857,27119.808088,45835.150384,3.269047,11.062307,-0.002117,25763.817684
7,A6,D2,84285.0,BBC,44660.7,34570.355469,0.242987,29974.215713,53982.855654,3.135280,9.927888,-0.002233,28475.504927
8,A6,D2,101025.0,BBC,61502.0,27689.375000,0.313347,49267.649260,348444.606763,5.562883,33.811002,-0.000688,46804.266797
9,A6,D2,247680.0,BBC,54266.5,116964.093750,0.116570,37825.596677,109524.712272,3.034175,8.522778,-0.002105,35934.316844


In [122]:
annot_features_yes = pd.DataFrame()
annot_features_no = pd.DataFrame()

for path in path_names:
    
    annot_yes, annot_no = create_annotation_slice_features(path, annot_data, size_random = 50)
    
    annot_features_yes = annot_features_yes.append(annot_yes, ignore_index=True)
    annot_features_no = annot_features_no.append(annot_no, ignore_index=True)

#create and save combined dataframe of yes and no
annot_features_yes.drop(['Unnamed: 0', 'High Freq (Hz)'], axis=1, inplace=True)
annot_features_full = pd.concat([annot_features_yes, annot_features_no])
print(annot_features_full.shape)
print(annot_features_full.Annotation.value_counts())

annot_features_full.to_csv('annot_features_full_pain.csv')

A2 D2
A2 D2
(13333,)
13333
A4 D2
A4 D2
(13333,)
13333
A6 D2
A6 D2
(13333,)
13333
A8 D2
A8 D2
(13333,)
13333
A10 D2
A10 D2
(13333,)
13333
A12 D2
A12 D2
(13333,)
13333
A2 D3
A2 D3
(13333,)
13333
A4 D3
A4 D3
(13333,)
13333
A6 D3
A6 D3
(13333,)
13333
A8 D3
A8 D3
(13333,)
13333
A10 D3
A10 D3
(13333,)
13333
A12 D3
A12 D3
(13333,)
13333
A2 D4
A2 D4
(13333,)
13333
A4 D4
A4 D4
(13333,)
13333
A6 D4
A6 D4
(13333,)
13333
A8 D4
A8 D4
(13333,)
13333
A10 D4
A10 D4
(13333,)
13333
A12 D4
A12 D4
(13333,)
13333
(1078, 12)
rand_noise    900
BBC           178
Name: Annotation, dtype: int64


In [123]:
annot_features_full.head()

,animal_number,session,time_stamp,Annotation,power_sum,spec_pur,spec_cent,spec_spread,spec_skew,spec_kurt,spec_slope,spec_roll
0,A2,D2,32692.5,BBC,33001.679688,0.278688,35066.527850,44975.041894,2.700772,6.672909,-0.001613,33313.201458
1,A2,D2,97515.0,BBC,42889.351562,0.209728,33325.032145,64997.459077,2.717043,6.765939,-0.002337,31658.780537
2,A2,D2,124942.5,BBC,32306.705078,0.250006,30640.698606,49617.524980,2.513463,5.396313,-0.002205,29108.663676
3,A2,D2,292027.5,BBC,128251.039062,0.089902,37497.496525,201737.430661,3.751878,12.999226,-0.002475,35622.621699
4,A2,D2,296505.0,BBC,65412.070312,0.180102,35595.230449,62452.123418,3.204285,11.323622,-0.002025,33815.468927
